In [15]:
import pandas as pd
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import os

import pytorch_lightning as pl
import torch.nn.functional as F
from torch import nn
from torch.optim import Adam
from torchvision.models import resnet50, ResNet50_Weights
import torch
from PIL import Image
import multiprocessing

num_cpu_cores = multiprocessing.cpu_count()
num_cpu_cores

6

In [2]:
# to read .csv files from another directory
location = "detect-pneumonia-spring-2024/" # "/<path>"

df = pd.read_csv(location + "labels_train.csv")

# Get the count of images in each class
class_counts = df['class_id'].value_counts()

# Define the target number of images per class
target_images_per_class = 10000

# Calculate the number of additional images needed for each class
additional_images_needed = target_images_per_class - class_counts
print(additional_images_needed)

class_id
1    7762
0    8773
2    8793
Name: count, dtype: int64


In [3]:
df

,file_name,class_id
0,img_4358977458434011046.jpg,1
1,img_5224016757187192130.jpg,2
2,img_3065202206106254707.jpg,2
3,img_6304894865561547174.jpg,1
4,img_3371338542810939877.jpg,2
...,...,...
4667,img_6661311872293090412.jpg,2
4668,img_3844568579349757418.jpg,1
4669,img_9145812369383814369.jpg,1
4670,img_1311393330250392648.jpg,1


In [4]:
# Define the augmentation function using OpenCV
def augment_image(image):
    # Random rotation
    angle = np.random.uniform(-10, 10)
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Random width and height shift
    tx = np.random.uniform(-0.1, 0.1) * image.shape[1]
    ty = np.random.uniform(-0.1, 0.1) * image.shape[0]
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Random shear
    shear_factor = np.random.uniform(-0.2, 0.2)
    M = np.float32([[1, shear_factor, 0], [0, 1, 0]])
    image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

    # Random zoom
    zx, zy = np.random.uniform(0.9, 1.1, 2)
    image = cv2.resize(image, None, fx=zx, fy=zy, interpolation=cv2.INTER_LINEAR)

    # Random horizontal flip
    if np.random.rand() < 0.5:
        image = cv2.flip(image, 1)
    
    return image

# Define the custom dataset
class PneumoniaDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, target_images_per_class=10000):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_images_per_class = target_images_per_class
        
        # Calculate the number of images needed for each class
        self.class_counts = self.annotations['class_id'].value_counts()
        self.additional_images_needed = target_images_per_class - self.class_counts

        # Create lists of images per class
        self.image_paths_by_class = {class_id: [] for class_id in self.additional_images_needed.index}
        for idx, row in self.annotations.iterrows():
            self.image_paths_by_class[row['class_id']].append(row['file_name'])
        
    def __len__(self):
        return self.target_images_per_class * len(self.additional_images_needed)

    def __getitem__(self, idx):
        # Determine the class and the specific image index within that class
        class_id = idx // self.target_images_per_class
        img_idx = idx % self.target_images_per_class
        
        if img_idx < self.class_counts[class_id]:
            # Original image
            img_name = self.image_paths_by_class[class_id][img_idx]
            img_path = os.path.join(self.root_dir, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        else:
            # Augmented image
            img_name = np.random.choice(self.image_paths_by_class[class_id])
            img_path = os.path.join(self.root_dir, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = augment_image(image)
        
        if self.transform:
            image = self.transform(image)
        
        label = class_id
        
        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create the dataset and dataloader
dataset = PneumoniaDataset(
    csv_file= location + 'labels_train.csv',
    root_dir= location + 'train_images/train_images',
    transform=transform,
    target_images_per_class=10000
)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers= num_cpu_cores)

# Example of iterating through the dataloader
for images, labels in dataloader:
    print(images.shape, labels.shape)
    break

torch.Size([32, 1, 224, 224]) torch.Size([32])


In [5]:
# def augment_image(image):
#     # Random rotation
#     angle = np.random.uniform(-10, 10)
#     M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
#     image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

#     # Random width and height shift
#     tx = np.random.uniform(-0.1, 0.1) * image.shape[1]
#     ty = np.random.uniform(-0.1, 0.1) * image.shape[0]
#     M = np.float32([[1, 0, tx], [0, 1, ty]])
#     image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

#     # Random shear
#     shear_factor = np.random.uniform(-0.2, 0.2)
#     M = np.float32([[1, shear_factor, 0], [0, 1, 0]])
#     image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))

#     # Random zoom
#     zx, zy = np.random.uniform(0.9, 1.1, 2)
#     image = cv2.resize(image, None, fx=zx, fy=zy, interpolation=cv2.INTER_LINEAR)

#     # Random horizontal flip
#     if np.random.rand() < 0.5:
#         image = cv2.flip(image, 1)
    
#     return image


# def resize_image(image, target_width=224, target_height=224):
#     return cv2.resize(image, (target_width, target_height))

In [6]:
# class PneumoniaDataset(Dataset):
#     def __init__(self, csv_file, root_dir, transform=None):
#         self.annotations = pd.read_csv(csv_file)
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.annotations)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
#         image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        
#         if self.transform:
#             image = self.transform(image)
        
#         # Convert image to 3-channel (if necessary) for pre-trained models
#         image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
#         image = resize_image(image)
        
#         label = int(self.annotations.iloc[idx, 1])
#         image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        
#         return image, label

# def augment_and_resize(image):
#     augmented_image = augment_image(image)
#     resized_image = resize_image(augmented_image)
#     return resized_image

In [7]:
# # Apply transformations using the function
# dataset = PneumoniaDataset(csv_file= location +'labels_train.csv', 
#                            root_dir= location + 'train_images', 
#                            transform=augment_and_resize)

# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [16]:
class PneumoniaModel(pl.LightningModule):
    def __init__(self, num_classes=3):
        super(PneumoniaModel, self).__init__()
        self.model = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-4)

In [ ]:
model = PneumoniaModel()

trainer = pl.Trainer(max_epochs=10, accelerator='dp') if torch.cuda.is_available() else pl.Trainer(max_epochs=10)
trainer.fit(model, dataloader)

In [ ]:
# Specify the directory and file name to save the model
model_save_path = location + "pneumonia_model_v1.pth"

# Save the trained model
torch.save(model.state_dict(), model_save_path)


In [ ]:
# Define the transform to preprocess the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define a function to predict the class for a single image
def predict_image(model, image_path, transform):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient tracking during inference
        outputs = model(image)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Get predicted label
    return predicted.item()

# Load the trained model
model = torch.load(model_save_path)  # Load your trained model

# Path to the folder containing testing images
test_folder = location + 'test_images/test_images'

# Iterate over testing images and make predictions
predictions = []
file_names = []
for file_name in os.listdir(test_folder):
    image_path = os.path.join(test_folder, file_name)
    predicted_label = predict_image(model, image_path, transform)
    predictions.append(predicted_label)
    file_names.append(file_name)

# Create a DataFrame with file names and predicted labels
results_df = pd.DataFrame({'file_name': file_names, 'class_id': predictions})

# Save the DataFrame to a CSV file
results_df.to_csv('labels_test.csv', index=False)